In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

invoices_id = "1_JUP1EYc4r1jgNpeLh7mMQxsKOsT-pcs"
customers_id = "1n6ePaU-cPfzhBKnSeGeynAxR9IYmCS0w"

invoices = pd.read_csv(f"https://drive.google.com/uc?id={invoices_id}")

url = f"https://drive.usercontent.google.com/download?id={customers_id}&export=download&authuser=0&confirm=t"
response = requests.get(url)
data = json.loads(response.text)

customers = pd.DataFrame.from_dict(data, orient='index').reset_index()
customers = customers.rename(columns={'index': 'customer_id'})
customers['customer_id'] = pd.to_numeric(customers['customer_id'], errors='coerce')
customers['age'] = pd.to_numeric(customers.get('age', pd.NA), errors='coerce')

invoices['invoice_date'] = pd.to_datetime(invoices['invoice_date'], errors='coerce')
invoices['price'] = pd.to_numeric(invoices['price'], errors='coerce')

df = invoices.merge(customers, on='customer_id', how='left')

In [3]:
# Easy 1
invoices['customer_id'].nunique()

5191

In [4]:
# Easy 2  
df['category'].unique()
df['category'].nunique()

8

In [5]:
# Easy 3
df['payment_method'].value_counts().head(1)

payment_method
Cash    44447
Name: count, dtype: int64

In [6]:
# Medium 1
df.groupby('category')['price'].sum().sort_values(ascending=False).head(3)

category
Clothing      31075684.64
Shoes         18135336.89
Technology    15772050.00
Name: price, dtype: float64

In [7]:
# Medium 2
df[df['age'] > 45]['price'].sum()

0.0

In [ ]:
# Medium 3
monthly_sales = df.set_index('invoice_date')['price'].resample('M').sum()
monthly_sales.plot(figsize=(10,6))
plt.title('Monthly Sales Over Time')
plt.show()

In [ ]:
# Hard 1
age_bins = list(range(0, 101, 10))
age_labels = [f"{i}-{i+9}" for i in range(0, 100, 10)]
df['age_decade'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)
pd.pivot_table(df, values='price', index=['category', 'age_decade'], aggfunc='sum', fill_value=0)